In [ ]:
import h5py
import ipydatawidgets
import ipywidgets
import numpy as np
import traitlets
from IPython.display import DisplayObject, display

import widgyts
import yt

yt.funcs.mylog.setLevel(0)

In [ ]:
ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030")

In [ ]:
class State(traitlets.HasTraits):
    sl = traitlets.Instance(yt.data_objects.selection_data_containers.YTSlice)
    im = traitlets.Instance(ipywidgets.Image)
    width = traitlets.Float(1.0)
    center_x = traitlets.Float(0.5)
    center_y = traitlets.Float(0.5)
    frb = traitlets.Instance(yt.FixedResolutionBuffer)

    @traitlets.observe("center_x", "center_y")
    def on_change_center(self, event):
        self.frb = self.sl.to_frb(
            self.width, (512, 512), center=(self.center_x, self.center_y)
        )

    @traitlets.observe("width")
    def on_change_width(self, event):
        self.frb = self.sl.to_frb(
            self.width, (512, 512), center=(self.center_x, self.center_y)
        )

    @traitlets.observe("frb")
    def on_change_frb(self, event):
        self.im.value = yt.write_bitmap(
            yt.write_image(np.log10(self.frb["density"]), None), None
        )


# ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030")
im = ipywidgets.Image(width=512, height=512)
s = State(sl=ds.r[:, :, 0.5], im=im)
width_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, value=1.0, step=0.001)
center_x_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, value=0.5, step=0.001)
center_y_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, value=0.5, step=0.001)
traitlets.link((width_slider, "value"), (s, "width"))
traitlets.link((center_x_slider, "value"), (s, "center_x"))
traitlets.link((center_y_slider, "value"), (s, "center_y"))

display(ipywidgets.VBox([im, width_slider, center_x_slider, center_y_slider]))

In [ ]:
slc = ds.r[:, :, 0.5]
frb = widgyts.FRBViewer(
    height=512,
    width=512,
    px=slc["px"],
    py=slc["py"],
    pdx=slc["pdx"],
    pdy=slc["pdy"],
    val=slc["density"],
    layout=ipywidgets.Layout(width="512px", height="512px"),
)
controls = frb.setup_controls()
display(ipywidgets.HBox([controls, frb]))

In [ ]:
frb.view_center

In [ ]:
from time import process_time

In [ ]:
def time_widgyts(spaces, n):
    times = np.array([])
    for i in range(n):
        start = process_time()
        for space in spaces:
            frb.view_center = (space, 0.5)
        end = process_time()
        time = end - start
        times = np.append(times, [time])
    avg_time = np.mean(times)
    time_std = np.std(times)
    err = time_std / (n**0.5)
    return avg_time, err

In [ ]:
def time_jupyter(spaces, n):
    times = np.array([])
    for i in range(n):
        start = process_time()
        for space in spaces:
            s.center_x = space
        end = process_time()
        time = end - start
        times = np.append(times, [time])
    avg_time = np.mean(times)
    time_std = np.std(times)
    err = time_std / (n**0.5)
    return avg_time, err

In [ ]:
num_events = 100  # highest number of events that will be timed
n = 10  # number of times the loop will be repeated for statistical purposes

# create empty arrays for timing and errors
widgyt_times = np.array([])
widgyt_err = np.array([])
jupyter_times = np.array([])
jupyter_err = np.array([])

for events in np.arange(5, num_events + 1, 5):
    # reset the center from the previous set of events
    center = 0.5
    s.center_x = center
    frb.view_center = (center, 0.5)

    # get an array of the x values that will be iterated over
    spaces = np.arange(1, events) * (1.0 / num_events)

    # get the average time and standard error for the timing results
    time_w1, err_w1 = time_widgyts(spaces, n)
    time_w2, err_w2 = time_jupyter(spaces, n)

    # add timing values and error values to arrays
    widgyt_times, widgyt_err = np.append(widgyt_times, [time_w1]), np.append(
        widgyt_err, [err_w1]
    )
    jupyter_times, jupyter_err = np.append(jupyter_times, [time_w2]), np.append(
        jupyter_err, [err_w2]
    )
    print(
        "number of events:",
        events,
        "widgyts time:",
        time_w1,
        "+-",
        err_w1,
        "jupyter time:",
        time_w2,
        "+-",
        err_w2,
    )

In [ ]:
x = np.arange(5, num_events + 1, 5)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams["text.usetex"] = True
matplotlib.rcParams["font.family"] = "STIXGeneral"

In [ ]:
tval = 1.833  # this is a value to get the 95% confidence value https://en.wikipedia.org/wiki/Student%27s_t-distribution
fig, ax = plt.subplots()
ax.errorbar(
    x,
    widgyt_times,
    color="#E9B106",
    yerr=tval * widgyt_err,
    alpha=0.6,
    label="widgyts",
    fmt="o",
)
ax.errorbar(
    x,
    jupyter_times,
    color="#369DB4",
    yerr=tval * jupyter_err,
    alpha=0.6,
    label="jupyter",
    fmt="o",
)
ax.legend()
ax.set_xlabel(r"Number of Interactions")
ax.set_ylabel(r"Time (s) to Complete Interactions")
ax.set_yscale("log")
ax.set_ylim(bottom=0.001)

In [ ]:
fig.savefig("timing.png", bbox_inches="tight", dpi=200)